In [1]:
import numpy as np
import random
import pdb
import pandas as pd

In [63]:
dataset = pd.read_excel('./AirQualityUCI.xlsx')
columns = dataset.columns

In [64]:
data_ip = dataset[[columns[3],columns[6],columns[8],columns[10],columns[11],columns[12],columns[13],columns[14]]]
desire_op = dataset[[columns[5]]]

In [20]:
class Particle():
    def __init__(self,layer):
        self.layer = []
        self.velocity = []
        self.weightmin = -1
        self.weightmax = 1
        self.fitness = 0
        self.position = []
        self.getweight(layer)
        self.pbest_position = self.position
        self.pbest_value = float('inf')


    def getweight(self,layer):
        self.layer = layer 
        chromolen = 0
        
        for i in range(1,len(layer)):
            chromolen = chromolen +  (layer[i-1]*layer[i])
        self.velocity = np.zeros(chromolen)
#         print(self.velocity)
        
        gene = np.zeros(chromolen)
        for j in range(len(gene)):
            gene[j] = self.weightmin + (self.weightmax - self.weightmin)*random.random()
        self.position.append(gene)
    
    def move(self):
        self.position = self.position + self.velocity

In [90]:
class Space():

    def __init__(self, target, n_particles,train_ip,train_op,layer):
        self.target = target
        self.n_particles = n_particles
        self.particles = []
        self.gbest_value = float('inf')        
        self.weightmin = -1
        self.weightmax = 1
        self.layer = layer
        self.gbest_position = []
        self.get_gbest(self.layer)
        self.train_i = train_ip
        self.train_o = train_op

#         print(self.gbest_position)
    
    def get_gbest(self,layer):
        chromolen = 0
        
        for i in range(1,len(layer)):
            chromolen = chromolen +  (layer[i-1]*layer[i])
        
        gene = np.zeros(chromolen)
        for j in range(len(gene)):
            gene[j] = self.weightmin + (self.weightmax - self.weightmin)*random.random()
        self.pogbest_positionsition = gene
        
    def set_pbest(self):
        for particle in self.particles:
#             print(particle.position)
            self.computefitness(particle)
#             print(particle.fitness)
#             fitness_cadidate = self.fitness(particle)
            if(particle.pbest_value > particle.fitness):
                particle.pbest_value = particle.fitness
                particle.pbest_position = particle.position
            print('----> pbest : ',particle.pbest_value)

    def set_gbest(self):
        for particle in self.particles:
            self.computefitness(particle)
#             best_fitness_cadidate = self.fitness(particle)
#             print(particle.fitness)
            if(self.gbest_value > particle.fitness):
                self.gbest_value = particle.fitness
                self.gbest_position = particle.position

    def move_particles(self):
        for particle in self.particles:
            global W
#             pdb.set_trace()
            new_velocity = (W*particle.velocity)  + (c1*random.random()) * (np.array(particle.pbest_position) - np.array(particle.position)) + (random.random()*c2) * (np.array(self.gbest_position) - np.array(particle.position))
            particle.velocity = new_velocity
            particle.move()
    
    def computefitness(self,particle):
        mlp = MLP(self.layer)
        mlp.initweight(particle.position)
        particle.fitness = 0
        for i in range(len(self.train_i)) :
            error = mlp.forward(self.train_i.iloc[i],self.train_o.iloc[i])
            particle.fitness += error
        particle.fitness = particle.fitness/len(self.train_i)
#         print(chomosome.fitness)


In [88]:
class MLP:
    def __init__(self,structure_layer):
        self.maxNode = 0
        self.structure = structure_layer
        
        for i in structure_layer:
            if self.maxNode < i :
                self.maxNode = i 
                
        self.weight = np.zeros((len(structure_layer)-1,self.maxNode,self.maxNode))
#         print(self.weight)

    def initweight(self,chomosome):
        count = 0
#         print(self.weight)
        for i in range(len(self.structure)-1):
            for j in range(self.structure[i]):
                for k in range(self.structure[i+1]):
#                     print(chomosome.position[count])
#                     print(chomosome[count])
                    self.weight[i][j][k] = chomosome[0][count]
                    count = count+1
        
    def forward(self,data,op):
        y = np.zeros((len(self.structure),self.maxNode))
#         print(list(data.values())[2:])
        for i in range(len(data.values)):
            y[0][i] = data.values[i]
        for i in range(1,len(self.structure)):
            for j in range(self.structure[i]):
                v = 0
                for k in range(self.structure[i-1]):
                    v = v + y[i-1][k]*self.weight[i-1][k][j]
                y[i][j] = self.sigmoid(v)  
#         print(abs(op.values-y[len(self.structure)-1][0]))        
        return abs(op.values-y[len(self.structure)-1][0])
               
    def sigmoid(self,v):
        return 1/(1+np.exp(-v))

In [94]:
import random
import numpy as np 


c1 = 1.2
c2 = 1.4 
W = (1/2*(c1+c2)-1)+0.3

n_iterations = 10
n_particles = 10
structure = [8,15,1]  

for c in range(10):
    print("------------------------- # Round "+str(c)+' ------------------------- ')
    i = int(c*len(data_ip)*0.1)
    test_ip = data_ip[i:int(i+int(len(data_ip)*0.1))]
    test_op = desire_op[i:int(i+int(len(data_ip)*0.1))]
    train_ip = data_ip.drop(data_ip.index[i:i+int(len(data_ip)*0.1)])
    train_ip = desire_op.drop(desire_op.index[i:i+int(len(data_ip)*0.1)])
    
    particles_vector = []
    search_space = Space(1, n_particles,train_ip,desire_op,structure)
    particles_vector = [Particle(structure) for _ in range(search_space.n_particles)]
    search_space.particles = particles_vector
    iteration = 0
    while(iteration < n_iterations):
        print("iteration : #"+str(iteration+1))
        search_space.set_pbest()    
        search_space.set_gbest()
        search_space.move_particles()
        iteration += 1
        print("-----> gbest : ", search_space.gbest_value[0])

    print("The best solution is: ", search_space.gbest_position, " in n_iterations: ", iteration)

------------------------- # Round 0 ------------------------- 
iteration : #1
----> pbest :  [18.27730955]
----> pbest :  [17.95923904]
----> pbest :  [18.30713611]
----> pbest :  [18.23726902]
----> pbest :  [18.05465534]
----> pbest :  [18.11873729]
----> pbest :  [17.77254546]
----> pbest :  [17.80070293]
----> pbest :  [17.73551307]
----> pbest :  [17.88641468]
-----> gbest :  17.73551306923878
iteration : #2
----> pbest :  [18.00504504]
----> pbest :  [17.70118516]
----> pbest :  [18.24889338]
----> pbest :  [17.86467721]
----> pbest :  [17.98308353]
----> pbest :  [17.68804334]
----> pbest :  [17.73949674]
----> pbest :  [17.77042339]
----> pbest :  [17.73551307]
----> pbest :  [17.71189637]
-----> gbest :  17.68804334096985
iteration : #3
----> pbest :  [17.68087041]
----> pbest :  [17.66679377]
----> pbest :  [17.67058493]
----> pbest :  [17.65857392]
----> pbest :  [17.67727925]
----> pbest :  [17.65692894]
----> pbest :  [17.68930754]
----> pbest :  [17.67380425]
----> pbest 

c:\users\thanakom_hatsadeang\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in exp


----> pbest :  [17.64195631]
----> pbest :  [17.64155687]
----> pbest :  [17.64189554]
----> pbest :  [17.64250677]
----> pbest :  [17.65023377]
----> pbest :  [17.64317431]
----> pbest :  [17.64386409]
----> pbest :  [17.64144365]
----> pbest :  [17.64234474]
----> pbest :  [17.64599498]
-----> gbest :  17.641443645988616
iteration : #7
----> pbest :  [17.64126285]
----> pbest :  [17.64114101]
----> pbest :  [17.64134658]
----> pbest :  [17.64178764]
----> pbest :  [17.641253]
----> pbest :  [17.64144883]
----> pbest :  [17.6414649]
----> pbest :  [17.64117286]
----> pbest :  [17.64120542]
----> pbest :  [17.64121509]
-----> gbest :  17.64114101258304
iteration : #8
----> pbest :  [17.64104419]
----> pbest :  [17.64108373]
----> pbest :  [17.64111944]
----> pbest :  [17.64105476]
----> pbest :  [17.64105765]
----> pbest :  [17.64100653]
----> pbest :  [17.64104302]
----> pbest :  [17.64106897]
----> pbest :  [17.64100122]
----> pbest :  [17.64113384]
-----> gbest :  17.641001220828624

----> pbest :  [17.75909985]
----> pbest :  [18.28472783]
----> pbest :  [17.88229702]
----> pbest :  [18.31597029]
----> pbest :  [18.40704041]
----> pbest :  [18.20278198]
----> pbest :  [18.37368425]
----> pbest :  [18.24706916]
----> pbest :  [18.36295597]
----> pbest :  [18.45289185]
-----> gbest :  17.759099846327473
iteration : #2
----> pbest :  [17.75909985]
----> pbest :  [17.80664259]
----> pbest :  [17.81284478]
----> pbest :  [18.03285404]
----> pbest :  [18.37389707]
----> pbest :  [17.70077899]
----> pbest :  [17.81713476]
----> pbest :  [17.9140161]
----> pbest :  [17.78154365]
----> pbest :  [18.4281594]
-----> gbest :  17.700778988380776
iteration : #3
----> pbest :  [17.69135145]
----> pbest :  [17.64712409]
----> pbest :  [17.66987286]
----> pbest :  [17.72494365]
----> pbest :  [17.95882321]
----> pbest :  [17.64890218]
----> pbest :  [17.66232729]
----> pbest :  [17.7337074]
----> pbest :  [17.74734584]
----> pbest :  [17.89195534]
-----> gbest :  17.64712408699811

----> pbest :  [17.64089297]
----> pbest :  [17.64089447]
----> pbest :  [17.64089325]
----> pbest :  [17.64089496]
----> pbest :  [17.64089295]
----> pbest :  [17.6408932]
-----> gbest :  17.6408925662653
iteration : #9
----> pbest :  [17.64089283]
----> pbest :  [17.64089256]
----> pbest :  [17.64089278]
----> pbest :  [17.64089257]
----> pbest :  [17.64089268]
----> pbest :  [17.64089256]
----> pbest :  [17.64089285]
----> pbest :  [17.64089267]
----> pbest :  [17.64089254]
----> pbest :  [17.64089268]
-----> gbest :  17.640892538794976
iteration : #10
----> pbest :  [17.64089274]
----> pbest :  [17.64089256]
----> pbest :  [17.64089253]
----> pbest :  [17.64089257]
----> pbest :  [17.64089266]
----> pbest :  [17.64089253]
----> pbest :  [17.64089272]
----> pbest :  [17.64089258]
----> pbest :  [17.64089253]
----> pbest :  [17.64089259]
-----> gbest :  17.640892525641473
The best solution is:  [[ -4.66551957   1.41594094  -0.47265235   6.56116705  -0.14317466
    3.93036002   6.3623

----> pbest :  [18.28181359]
----> pbest :  [18.0556011]
----> pbest :  [17.9860635]
----> pbest :  [18.2071488]
----> pbest :  [17.91968967]
----> pbest :  [18.45684628]
----> pbest :  [17.82994862]
-----> gbest :  17.82841538509035
iteration : #2
----> pbest :  [17.82841539]
----> pbest :  [18.03818321]
----> pbest :  [18.42776064]
----> pbest :  [17.76099943]
----> pbest :  [17.84551103]
----> pbest :  [17.86478554]
----> pbest :  [18.18080528]
----> pbest :  [17.75730773]
----> pbest :  [18.25840114]
----> pbest :  [17.80437105]
-----> gbest :  17.75730773311054
iteration : #3
----> pbest :  [17.82680026]
----> pbest :  [18.03818321]
----> pbest :  [18.32093675]
----> pbest :  [17.66597607]
----> pbest :  [17.81261972]
----> pbest :  [17.73486197]
----> pbest :  [17.72159358]
----> pbest :  [17.703717]
----> pbest :  [17.64881947]
----> pbest :  [17.80437105]
-----> gbest :  17.64881947342908
iteration : #4
----> pbest :  [17.66442335]
----> pbest :  [17.64387214]
----> pbest :  [1

-----> gbest :  17.642801254490845
iteration : #10
----> pbest :  [17.64280873]
----> pbest :  [17.64275162]
----> pbest :  [17.64275035]
----> pbest :  [17.64277472]
----> pbest :  [17.6427671]
----> pbest :  [17.64276837]
----> pbest :  [17.64275089]
----> pbest :  [17.64278538]
----> pbest :  [17.64275973]
----> pbest :  [17.64278559]
-----> gbest :  17.642750354575448
The best solution is:  [[-5.73246168e+00 -1.08900249e+00  4.84198272e+00 -1.86142782e+00
  -1.20572391e+00 -3.33061579e+00  2.11470993e+00  5.11536439e+00
   2.90978320e+00 -6.83201318e+00  3.22058901e+00 -2.21119173e+00
  -7.92375178e+00 -4.07072699e+00  4.63101822e+00  2.04281091e+00
   6.03534914e+00  6.68376145e+00 -4.52345008e-01  2.65821263e+00
   4.20743820e+00  3.36425366e+00 -7.36233483e-02  1.98658801e+00
  -3.58346843e+00 -4.93960815e+00 -5.48862999e+00 -6.99992842e+00
   1.26722481e+00  3.01517354e+00 -2.78265294e-01 -4.05561944e+00
   8.10911076e-01 -1.94007079e+00  2.83593803e-01  3.03318514e+00
  -6.219

----> pbest :  [18.05172767]
----> pbest :  [18.06236214]
----> pbest :  [18.17386748]
----> pbest :  [17.94466812]
----> pbest :  [17.92186455]
----> pbest :  [17.99291255]
----> pbest :  [17.74330248]
----> pbest :  [17.74285178]
----> pbest :  [18.10658901]
----> pbest :  [18.00482882]
-----> gbest :  17.74285178369634
iteration : #2
----> pbest :  [17.89884697]
----> pbest :  [18.06236214]
----> pbest :  [18.00337007]
----> pbest :  [17.8570837]
----> pbest :  [17.70871264]
----> pbest :  [17.95960503]
----> pbest :  [17.74330248]
----> pbest :  [17.74285178]
----> pbest :  [17.73321909]
----> pbest :  [17.89489057]
-----> gbest :  17.70871263853271
iteration : #3
----> pbest :  [17.73138554]
----> pbest :  [17.94997063]
----> pbest :  [17.73780029]
----> pbest :  [17.72536939]
----> pbest :  [17.62869019]
----> pbest :  [17.79813464]
----> pbest :  [17.67693362]
----> pbest :  [17.70421422]
----> pbest :  [17.63258874]
----> pbest :  [17.78576082]
-----> gbest :  17.62869019239244

----> pbest :  [17.59755525]
----> pbest :  [17.59755376]
----> pbest :  [17.59755393]
----> pbest :  [17.59755427]
----> pbest :  [17.59755447]
----> pbest :  [17.59755409]
----> pbest :  [17.59755388]
----> pbest :  [17.59755413]
----> pbest :  [17.59755427]
----> pbest :  [17.59755375]
-----> gbest :  17.59755374664829
The best solution is:  [[ 6.80737276 -2.11437633  3.17196963  2.48805499  0.3797138  10.40053631
   3.35312668  3.78197549 -2.66407665 -3.82888983 -1.31035187 -3.77676005
   2.49598597  0.02307615 -4.84367659 -2.21540779  3.03637541 -6.66366671
  -4.64884451  2.55199165  5.41232362 -3.13308382 -6.65237312 -1.70136404
  -6.72827768 -1.47692028 -2.62791252  3.58515988 -6.31104535  0.54532228
   1.52893395 -6.68131436  4.69983345 -3.58144719  0.55117385  0.44946702
  -0.30654302 -4.64021761 -2.96735615  6.52210466  3.06743663 -6.83764075
  -3.24470301  3.63351373  7.54443797 -3.15153284 -2.72623734 -8.66396405
   0.89326408 -6.38686688  5.09557132 -3.62683765 10.01910209

In [97]:
import random
import numpy as np 


c1 = 1.2
c2 = 1.4 
W = (1/2*(c1+c2)-1)+0.3

n_iterations = 10
n_particles = 10
structure = [8,15,1]  

for c in range(10):
    print("------------------------- # Round "+str(c)+' ------------------------- ')
    i = int(c*len(data_ip)*0.1)
    test_ip = data_ip[i:int(i+int(len(data_ip)*0.1))]
    test_op = desire_op[i:int(i+int(len(data_ip)*0.1))]
    train_ip = data_ip.drop(data_ip.index[i:i+int(len(data_ip)*0.1)])
    train_ip = desire_op.drop(desire_op.index[i:i+int(len(data_ip)*0.1)])
    
    particles_vector = []
    search_space = Space(1, n_particles,train_ip,desire_op,structure)
    particles_vector = [Particle(structure) for _ in range(search_space.n_particles)]
    search_space.particles = particles_vector
    iteration = 0
    while(iteration < n_iterations):
        print("iteration : #"+str(iteration+1))
        search_space.set_pbest()    
        search_space.set_gbest()
        search_space.move_particles()
        iteration += 1
        print("-----> gbest : ", search_space.gbest_value[0])

    print("The best solution is: ", search_space.gbest_position, " in n_iterations: ", iteration)

------------------------- # Round 0 ------------------------- 
iteration : #1
----> pbest :  [17.87372985]
----> pbest :  [17.78525689]
----> pbest :  [18.28568152]
----> pbest :  [18.02354381]
----> pbest :  [17.99631829]
----> pbest :  [17.71439598]
----> pbest :  [17.90514954]
----> pbest :  [18.18850479]
----> pbest :  [18.41260694]
----> pbest :  [17.95330838]
-----> gbest :  17.71439598208045
iteration : #2
----> pbest :  [17.71922416]
----> pbest :  [17.76584212]
----> pbest :  [18.0972294]
----> pbest :  [17.6857842]
----> pbest :  [17.84975013]
----> pbest :  [17.71439598]
----> pbest :  [17.85507597]
----> pbest :  [17.91081197]
----> pbest :  [17.70710475]
----> pbest :  [17.91958755]
-----> gbest :  17.685784201468355
iteration : #3
----> pbest :  [17.66079267]
----> pbest :  [17.76584212]
----> pbest :  [17.65235174]
----> pbest :  [17.66582928]
----> pbest :  [17.6716502]
----> pbest :  [17.7022003]
----> pbest :  [17.69209105]
----> pbest :  [17.73301249]
----> pbest :  

c:\users\thanakom_hatsadeang\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in exp


----> pbest :  [17.64176144]
----> pbest :  [17.64173741]
----> pbest :  [17.6430403]
-----> gbest :  17.641400589351786
iteration : #9
----> pbest :  [17.64202732]
----> pbest :  [17.64140059]
----> pbest :  [17.64430148]
----> pbest :  [17.6414339]
----> pbest :  [17.64181215]
----> pbest :  [17.64155795]
----> pbest :  [17.64148083]
----> pbest :  [17.64176144]
----> pbest :  [17.64120651]
----> pbest :  [17.64226463]
-----> gbest :  17.641206505120014
iteration : #10
----> pbest :  [17.64202732]
----> pbest :  [17.64123435]
----> pbest :  [17.6410828]
----> pbest :  [17.64115859]
----> pbest :  [17.64181215]
----> pbest :  [17.64124548]
----> pbest :  [17.64140727]
----> pbest :  [17.64145729]
----> pbest :  [17.64106528]
----> pbest :  [17.64160773]
-----> gbest :  17.641065275129634
The best solution is:  [[-2.63379499  4.41547364  1.09323919 -1.39342584  1.2516736   3.45462124
  -3.84950606  2.83257691  1.95573361  3.16776531 -3.40870592 -5.62595074
   3.5275277  -1.74886371 -0.

----> pbest :  [17.66024856]
----> pbest :  [17.73165339]
----> pbest :  [17.67633262]
----> pbest :  [17.74027669]
----> pbest :  [17.70427507]
----> pbest :  [17.67567514]
----> pbest :  [18.39506874]
----> pbest :  [18.26141616]
----> pbest :  [17.89738557]
----> pbest :  [17.6757008]
-----> gbest :  17.660248560080923
iteration : #3
----> pbest :  [17.64434123]
----> pbest :  [17.6541126]
----> pbest :  [17.6445339]
----> pbest :  [17.65110414]
----> pbest :  [17.64862939]
----> pbest :  [17.65958265]
----> pbest :  [17.65656304]
----> pbest :  [17.69933134]
----> pbest :  [17.64708335]
----> pbest :  [17.6757008]
-----> gbest :  17.644341232771954
iteration : #4
----> pbest :  [17.64232361]
----> pbest :  [17.64329963]
----> pbest :  [17.64165485]
----> pbest :  [17.64314547]
----> pbest :  [17.6423806]
----> pbest :  [17.64583471]
----> pbest :  [17.64174258]
----> pbest :  [17.64409519]
----> pbest :  [17.6412186]
----> pbest :  [17.66905561]
-----> gbest :  17.64121859927813
it

----> pbest :  [17.64090232]
----> pbest :  [17.64090923]
----> pbest :  [17.64091659]
-----> gbest :  17.640901993375405
The best solution is:  [[ -2.12943868  -3.53771998  -6.94670445   8.07053471   9.15331
    4.37419104   4.31474045   2.42697501  -0.29611292   3.78813626
   -0.10496676  -5.13782716   3.93512864  -1.08734151   1.8615294
   -4.27221334   2.13965082   0.08831879  -0.57989229   5.57475099
   -2.41673389  -6.62989587  -0.02007773   9.50753616   0.78407123
    1.19614905  -2.31241076  -4.79343439  -5.38992451   1.33392123
    7.7640855    3.5866218   -3.23381578  -4.12679276  -4.44238962
    9.49485299  -1.03952372  -1.34748516  -1.73383422   1.53985222
    0.78269456   1.39707127   0.65274521  -7.00374611   3.63796105
    1.09203792   3.68562928   4.93456545   0.69801604   7.33196372
   -2.34188527   6.72066387  -7.60452497  -2.56858503   1.07443698
    7.83001796  -1.14966917   0.11035113  -2.85408117   0.52710844
   -1.11330329   4.30351794  -6.54441977  -1.91105018  

----> pbest :  [17.86268853]
----> pbest :  [18.26696195]
----> pbest :  [17.67168644]
----> pbest :  [18.25602935]
----> pbest :  [17.95588994]
----> pbest :  [17.66499143]
----> pbest :  [17.77013484]
-----> gbest :  17.654712703051374
iteration : #3
----> pbest :  [17.64353718]
----> pbest :  [17.64186571]
----> pbest :  [17.65228374]
----> pbest :  [17.68132769]
----> pbest :  [17.64800654]
----> pbest :  [17.64164574]
----> pbest :  [17.64525141]
----> pbest :  [17.76475098]
----> pbest :  [17.6430629]
----> pbest :  [17.680333]
-----> gbest :  17.641645744700085
iteration : #4
----> pbest :  [17.64109661]
----> pbest :  [17.64110287]
----> pbest :  [17.64162046]
----> pbest :  [17.64117503]
----> pbest :  [17.64096911]
----> pbest :  [17.64102275]
----> pbest :  [17.64103867]
----> pbest :  [17.64114667]
----> pbest :  [17.64122051]
----> pbest :  [17.65273212]
-----> gbest :  17.640969114708284
iteration : #5
----> pbest :  [17.64093577]
----> pbest :  [17.64095529]
----> pbest 

----> pbest :  [17.6408926]
----> pbest :  [17.64089273]
----> pbest :  [17.64089296]
-----> gbest :  17.640892529589674
iteration : #10
----> pbest :  [17.64089252]
----> pbest :  [17.64089252]
----> pbest :  [17.64089253]
----> pbest :  [17.64089252]
----> pbest :  [17.64089254]
----> pbest :  [17.64089252]
----> pbest :  [17.64089252]
----> pbest :  [17.64089252]
----> pbest :  [17.64089253]
----> pbest :  [17.64089263]
-----> gbest :  17.64089251969414
The best solution is:  [[  2.43509292  -1.83663071  -2.3971459    0.74558339   3.73236351
    3.84846985   3.73569487  -2.54109208  -0.06362172   4.38643136
    0.42745483  -4.13576134 -12.25941057  -6.21192255  -5.50647104
    4.12439037  -1.24077674   2.49635704  -4.80589621  10.36192079
    1.82387767   5.08042529   7.42709412  -0.30239424   5.58594372
   -4.95233951 -10.0569457   -0.2389049   -2.57680622  -0.41764209
   -0.35134595  11.34002753   0.82219096  -6.40883369   0.96964011
  -11.91211276 -10.1517974   -1.89651018  -0.73

----> pbest :  [17.73325149]
----> pbest :  [17.67392002]
----> pbest :  [17.68025779]
----> pbest :  [17.80490356]
----> pbest :  [17.69097842]
----> pbest :  [17.7732354]
----> pbest :  [17.73184471]
----> pbest :  [17.66819638]
----> pbest :  [17.73193213]
----> pbest :  [17.76697089]
-----> gbest :  17.66819638090788
iteration : #4
----> pbest :  [17.69825749]
----> pbest :  [17.65544295]
----> pbest :  [17.64740412]
----> pbest :  [17.6537411]
----> pbest :  [17.67960587]
----> pbest :  [17.66828726]
----> pbest :  [17.66554035]
----> pbest :  [17.66454655]
----> pbest :  [17.70297339]
----> pbest :  [17.65013556]
-----> gbest :  17.647404120121596
iteration : #5
----> pbest :  [17.6552077]
----> pbest :  [17.64302547]
----> pbest :  [17.64219119]
----> pbest :  [17.63604793]
----> pbest :  [17.6614686]
----> pbest :  [17.63884548]
----> pbest :  [17.6456863]
----> pbest :  [17.64388172]
----> pbest :  [17.65606863]
----> pbest :  [17.63409969]
-----> gbest :  17.634099687521957
i

----> pbest :  [17.59755334]
----> pbest :  [17.59755435]
----> pbest :  [17.59755355]
----> pbest :  [17.59755338]
----> pbest :  [17.59755353]
----> pbest :  [17.59755333]
-----> gbest :  17.597553108240785
The best solution is:  [[ 3.68541099  1.17995523 -5.12649551  0.11856028  2.75701412 -6.4502716
   1.44270168  1.126745   -2.59511044 -3.80975741 -1.42845994 -1.978687
   0.15825789 -2.98585587  1.69582401 -3.28916841 -1.65539933 -1.48864844
   3.65394296 -5.76718161  0.50489541  7.18765295 -4.76069577  5.75811941
  -4.44817487 -2.90249523  2.65946154  0.56384006 -0.68676011 -2.25385067
  -2.99193343 -0.96730383 -2.73271029  1.29950323 -4.02721399  0.08016797
  -2.35228747 -1.2582923  -3.75538429 -3.31416941  1.93135994  3.51951414
  -1.25673079 -2.24857077 -4.93210906  5.27375485 -2.99159398  1.30159528
  -0.88151166 -2.51061344 -4.35162362  1.79887215  0.75202798 -1.64019652
  -3.9045928   2.80946466  6.52809062  3.2681188  -4.32540649  4.13081869
   1.93738199 -0.56949391  3.32